# Further Motion/Distance Analysis

We diver further into our motion analysis by exploring distance-related aspects of our tracking data




In [93]:
import os
import pandas as pd
from data_loading import load_tracking_data
import seaborn as sns
import matplotlib.pyplot as plt

In [94]:
# load tracking data for all weeks
root_dir = os.getcwd()
tracking_fname_list = [os.path.join(root_dir,  f"data/tracking_week_{i}.csv") for i in range(1,10)]
write_track = False
if write_track:
    df_tracking = pd.concat(
            [load_tracking_data(tracking_fname) for tracking_fname in tracking_fname_list]
        )
    df_tracking.to_csv('data/track_full.csv')
else: 
    df_tracking = pd.read_csv('data/track_full.csv',usecols=['gameId','playId','nflId','frameId','club','a','s','dis','y'])
df_players = pd.read_csv(os.path.join(root_dir,'data/players.csv'))
df_player_play = pd.read_csv(os.path.join(root_dir,'data/player_play.csv'))

Sub to offense this time:

In [95]:
plays_fname = os.path.join(root_dir,  "data/plays.csv")
df_plays = pd.read_csv(os.path.join(root_dir,plays_fname))
df_games = pd.read_csv(os.path.join(root_dir,'data/games.csv'))

In [96]:
play_trunc = df_plays[['gameId','playId','possessionTeam']]
pt_df = play_trunc.merge(df_tracking,how='left',left_on=['gameId','playId','possessionTeam'],
                 right_on=['gameId','playId','club'])

# na's dropped to lose players w/o tracking info
off_df = pt_df.dropna()

In [97]:
example = ((off_df['gameId'] == 2022102300) & (off_df['playId'] == 2314)) & (off_df['nflId'] == 42347)

### TODO: compare fastest vs second-fastest in last N seconds (10? 15?)

reincorp frame info, do last nm get nlargesst (2) etc

sort by keys and frame ig?

In [98]:
off_15 = off_df.groupby(['gameId','playId','nflId']).tail(15).reset_index(drop=True)

In [99]:
og_15 = off_15.groupby(['gameId','playId','nflId']).agg(y_initial=('y','first'),y_final=('y','last'),dis_sum=('dis','sum'),
                                                        s_max=('s','max'),a_max=('a','max'),a_mean=('a','mean'),a_median=('a','median'),
                                                        s_mean=('s','mean'),s_min=('s','min'),a_min=('a','min')).reset_index()

get y delta

In [100]:
og_15['y_delta'] = og_15['y_final']-og_15['y_initial']

In [101]:
og_15.columns[-4:]

Index(['s_mean', 's_min', 'a_min', 'y_delta'], dtype='object')

In [102]:
delta_15 = og_15[['gameId','playId']].drop_duplicates()

In [103]:
ser_ls = []

In [104]:
for col in og_15.columns[-10:]:

    ser_ls.append(og_15.groupby(['gameId','playId'])[col].apply(lambda grp: grp.nlargest(2).diff().tail(1)).reset_index(drop=True))

In [105]:
delta_15 = delta_15.reset_index(drop=True)

In [106]:
d15_proc = pd.concat([delta_15,pd.concat(ser_ls,axis=1)],axis=1)

In [107]:
d15_proc['y_delta'] = d15_proc['y_delta'].round(2)

In [108]:
d15_proc.head(10)

,gameId,playId,y_final,dis_sum,s_max,a_max,a_mean,a_median,s_mean,s_min,a_min,y_delta
0,2022090800,56,-9.29,-0.31,-0.06,-0.01,-0.028667,-0.02,-0.035333,-0.01,0.00,-0.00
1,2022090800,80,-4.48,-4.23,-4.73,-4.87,-4.250000,-4.48,-2.862667,-1.63,-2.95,-1.26
2,2022090800,101,-4.96,-1.33,-3.33,-3.15,-1.536000,-1.61,-1.054000,-0.03,-0.13,-1.55
3,2022090800,122,-5.15,-0.12,-0.02,-0.09,-0.018000,-0.02,-0.023333,-0.02,-0.02,-0.04
4,2022090800,167,-1.24,-0.23,-0.37,-0.92,-0.344000,-0.15,-0.241333,-0.04,-0.04,-0.01
5,2022090800,191,-5.07,-0.05,0.00,-0.03,-0.023333,-0.03,-0.056667,-0.01,-0.01,-0.02
6,2022090800,212,-10.81,-7.61,-6.72,-2.84,-2.090000,-2.26,-5.184000,-2.60,-0.32,-7.50
7,2022090800,236,-3.62,-3.66,-3.37,-3.77,-2.717333,-2.26,-2.727333,-0.36,-0.65,-0.02
8,2022090800,299,-6.86,-5.16,-6.38,-2.56,-2.988000,-2.95,-3.628667,-0.42,-1.46,-5.10
9,2022090800,343,-4.75,-3.61,-3.05,-3.03,-1.662000,-1.48,-2.444667,-0.67,-0.48,-3.67


In [109]:
d15_flagged = d15_proc.merge(df_plays[['gameId','playId','isDropback']],how='left')

In [110]:
d15_flagged['a_max'].quantile(.1)

-3.16

In [111]:
d15_flagged['amed_25'] = (d15_flagged['a_median'] >= d15_flagged['a_median'].quantile(.25))

In [112]:
d15_flagged['amean_dis'] = d15_flagged['a_mean']/.05*d15_flagged['dis_sum']

In [113]:
d15_flagged['amean_del'] =d15_flagged['a_mean']/.5*d15_flagged['y_delta']

In [114]:
d15_flagged.corr()['isDropback']

gameId       -0.022265
playId        0.013461
y_final      -0.101539
dis_sum       0.059853
s_max         0.120883
a_max         0.151071
a_mean        0.165415
a_median      0.162167
s_mean        0.062991
s_min        -0.028102
a_min         0.064790
y_delta       0.041878
isDropback    1.000000
amed_25       0.146124
amean_dis    -0.113067
amean_del    -0.075723
Name: isDropback, dtype: float64

TODO: incorp accel min, median/mean?

In [115]:
d15_flagged[['gameId','playId','a_mean','a_max']].to_csv('data/top_2_player_diff_last_15.csv')